# My Approach: Heuristic Intent Distribution Exploration
I need to get an insight into the true intents inside my Twitter data. Doing it by keyword might prove to be a good baseline way to do this. 


Using keywords as a starting point might offer a strong foundational approach for this task. I'm building upon this concept by employing heuristic clustering to organize my intents, aiming to minimize overlaps between them. The goal is to distill a set of intents that are both distinct and exclusive, enhancing Eve bot's ability to differentiate between them.

I drew inspiration from observing other solutions, particularly the "semantic fingerprint" concept implemented by cortex, although the specific workings were undisclosed. This prompted me to devise my approach. Initially, I considered branching off into manual selection of 1000 examples, but I realized this was impractical and overly laborious.

This notebook serves as my strategy for generating training data for intent classification.

In [ ]:
import pandas as pd
print(f'pandas: {pd.__version__}')
import numpy as np
print(f'numpy: {np.__version__}')
# Visualization 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
# Making my visualizations pretty
sns.set_style('whitegrid')
# Combination exploration
import itertools
import yaml

# Loading back processed data
processed = pd.read_pickle('objects/processed.pkl') # Load data from 1.0_ipynb file 
print(f'\ninbound:\n{processed.head()}')

## Brief Keyword Search EDA
Using this as a tool to look at Tweets.

In [ ]:
# Search by keywords (single keyword filter)
keyword = 'info'

# Seeing what the processed Tweets look like
filt = [(i,j) for i,j in enumerate(processed['Processed Inbound']) if keyword in j]
filtered = processed.iloc[[i[0] for i in filt]]
print(f'{len(filtered)} Tweets contain the keyword {keyword}')
filtered